In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# TTS (Text-To-Speech)

- TTS 모델은 텍스트를 자연스러운 음성으로 변환하는 AI 모델이다.
    - https://platform.openai.com/docs/models
    - tts-1 : 실시간 텍스트-음성 변환에 최적화된 최신 모델로 속도에 중점. 텍스트를 음성으로 빠르게 변환하는 기능 제공
    - tts-1-hd : 품질에 최적화된 최신 텍스트-음성 변환 모델로 높은 품질에 중점. 음성의 자연스러움과 선명도 강조

- 음성 선택지
    - https://platform.openai.com/docs/guides/text-to-speech#text-to-speech-models
    - Alloy: 부드럽고 자연스러운 톤의 음성
    - Echo: 명확하고 자신감 있는 음성
    - Fable: 이야기 전달에 적합한 서정적인 음성
    - Onyx: 전문적이고 신뢰감을 주는 음성
    - Nova: 활기차고 에너지 넘치는 음성
    - Shimmer: 부드럽고 진정시키는 음성


In [3]:
from openai import OpenAI

client = OpenAI()

text = "한번만, 딱 한 번만 말할거니까 잘들어. 너 좋아해. 니가 남자건 외계인이건 이제 상관 안해. 정리하는 거 힘들어서 못해먹겠으니까. 가보자 갈때까지. 한 번 가보자."

with client.audio.speech.with_streaming_response.create(
    model='tts-1',
    voice='coral',
    input=text
) as response:
    response.stream_to_file('tts_output.mp3')

---

# STT (Speech-To-Text)



- Whisper는 OpenAI에서 개발한 범용 음성 인식 모델로, 다양한 오디오 데이터셋을 학습하여 다국어 음성 인식, 음성 번역, 언어 식별 등의 작업을 수행할 수 있다.
- Whisper v2-large 모델은 현재 API를 통해 'whisper-1'이라는 이름으로 제공되고 있다.
    - https://platform.openai.com/docs/models/whisper-1
- 오픈 소스 버전의 Whisper와 API를 통한 Whisper는 기능적으로 동일하지만, API를 통해 제공되는 버전은 최적화된 추론 과정을 거쳐 다른 방법에 비해 더 빠르게 동작한다. 

In [4]:
with open("tts_output.mp3", "rb") as f:
    transcriptions = client.audio.transcriptions.create(
        model='whisper-1',
        file=f
    )

    print(transcriptions)

Transcription(text='한 번만 딱 한 번만 말할 거니까 잘 들어 너 좋아해 니가 남자건 외계인이건 이제 상관안해 정리하는거 힘들어서 못해먹겠으니까 가보자 갈 때까지 한번 가보자', logprobs=None, usage=UsageDuration(seconds=13.0, type='duration'))


---

# gTTS

In [5]:
!pip install gtts

  Using cached gTTS-2.5.4-py3-none-any.whl.metadata (4.1 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
Using cached gTTS-2.5.4-py3-none-any.whl (29 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)

   ---------------------------------------- 0/2 [click]
   ---------------------------------------- 0/2 [click]
   ---------------------------------------- 0/2 [click]
   -------------------- ------------------- 1/2 [gtts]
   -------------------- ------------------- 1/2 [gtts]
   -------------------- ------------------- 1/2 [gtts]
   -------------------- ------------------- 1/2 [gtts]
   ---------------------------------------- 2/2 [gtts]



In [6]:
from gtts import gTTS

google_tts = gTTS(
    text="안녕하세요, 저는 다람쥐입니다. 여러분 행복하세요~!",
    lang="ko"
)

google_tts.save('gtts_output.mp3')

---

# SpeechRecognition

- ffmpeg 설치 필요! (mp3 > wav 파일 변환)

In [7]:
!pip install SpeechRecognition

  Using cached speechrecognition-3.14.3-py3-none-any.whl.metadata (30 kB)
Using cached speechrecognition-3.14.3-py3-none-any.whl (32.9 MB)


In [9]:
!pip install pydub pyaudio

  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached PyAudio-0.2.14-cp312-cp312-win_amd64.whl.metadata (2.7 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
Using cached PyAudio-0.2.14-cp312-cp312-win_amd64.whl (164 kB)

   ---------------------------------------- 0/2 [pydub]
   -------------------- ------------------- 1/2 [pyaudio]
   ---------------------------------------- 2/2 [pyaudio]



In [ ]:
import speech_recognition as sr

recognizer = sr.Recognizer()

while True:
    with sr.Microphone() as source:
        print('말씀하세요.')
        audio = recognizer.listen(source)
        txt = recognizer.recognize_google(audio, language='ko-KR')
        print(txt)

        if txt == '종료':
            break

In [ ]:
from pydub import AudioSegment
import speech_recognition as sr

# mp3 -> wav 파일 변환
audio = AudioSegment.from_mp3('gtts_output.mp3')
audio.export('gtts_output_wav.wav', format='wav')

# 오디오 파일 로드
r = sr.Recognizer()
input_audio = sr.AudioFile('gtts_output_wav.wav')

# 오디오 데이터 객체 생성
with input_audio as source:
    audio = r.record(source)

# 텍스트 변환
result_txt = r.recognize_google(audio_data=audio, language='ko-KR')
print(result_txt)

안녕하세요 저는 다람쥐입니다 여러분 행복하세요 물결표
